<a href="https://colab.research.google.com/github/mossaudi/AES/blob/master/all_rag_techniques/simple_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple RAG (Retrieval-Augmented Generation) System

## Overview

This code implements a basic Retrieval-Augmented Generation (RAG) system for processing and querying PDF documents. The system encodes the document content into a vector store, which can then be queried to retrieve relevant information.

## Key Components

1. PDF processing and text extraction
2. Text chunking for manageable processing
3. Vector store creation using [FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/) and OpenAI embeddings
4. Retriever setup for querying the processed documents
5. Evaluation of the RAG system

## Method Details

### Document Preprocessing

1. The PDF is loaded using PyPDFLoader.
2. The text is split into chunks using RecursiveCharacterTextSplitter with specified chunk size and overlap.

### Text Cleaning

A custom function `replace_t_with_space` is applied to clean the text chunks. This likely addresses specific formatting issues in the PDF.

### Vector Store Creation

1. OpenAI embeddings are used to create vector representations of the text chunks.
2. A FAISS vector store is created from these embeddings for efficient similarity search.

### Retriever Setup

1. A retriever is configured to fetch the top 2 most relevant chunks for a given query.

### Encoding Function

The `encode_pdf` function encapsulates the entire process of loading, chunking, cleaning, and encoding the PDF into a vector store.

## Key Features

1. Modular Design: The encoding process is encapsulated in a single function for easy reuse.
2. Configurable Chunking: Allows adjustment of chunk size and overlap.
3. Efficient Retrieval: Uses FAISS for fast similarity search.
4. Evaluation: Includes a function to evaluate the RAG system's performance.

## Usage Example

The code includes a test query: "What is the main cause of climate change?". This demonstrates how to use the retriever to fetch relevant context from the processed document.

## Evaluation

The system includes an `evaluate_rag` function to assess the performance of the retriever, though the specific metrics used are not detailed in the provided code.

## Benefits of this Approach

1. Scalability: Can handle large documents by processing them in chunks.
2. Flexibility: Easy to adjust parameters like chunk size and number of retrieved results.
3. Efficiency: Utilizes FAISS for fast similarity search in high-dimensional spaces.
4. Integration with Advanced NLP: Uses OpenAI embeddings for state-of-the-art text representation.

## Conclusion

This simple RAG system provides a solid foundation for building more complex information retrieval and question-answering systems. By encoding document content into a searchable vector store, it enables efficient retrieval of relevant information in response to queries. This approach is particularly useful for applications requiring quick access to specific information within large documents or document collections.

# Package Installation and Imports

The cell below installs all necessary packages required to run this notebook.


In [15]:
# Install required packages
!pip install pypdf==5.6.0
!pip install PyMuPDF==1.26.1
!pip install python-dotenv==1.1.0
!pip install rank_bm25==0.2.2
!pip install faiss-cpu==1.11.0
!pip install deepeval
!pip install langchain langchain-community langchain-openai langchain-cohere chromadb
!pip install sentence-transformers

In [16]:
# Clone the repository to access helper functions and evaluation modules
!git clone https://github.com/NirDiamant/RAG_TECHNIQUES.git
import sys
sys.path.append('RAG_TECHNIQUES')

# If you need to run with the latest data
# !cp -r RAG_TECHNIQUES/data .

fatal: destination path 'RAG_TECHNIQUES' already exists and is not an empty directory.


In [18]:
import os
import sys
from dotenv import load_dotenv
from google.colab import userdata



# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable (comment out if not using OpenAI)
if not userdata.get('OPENAI_API_KEY'):
    os.environ["OPENAI_API_KEY"] = input("Please enter your OpenAI API key: ")
else:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Original path append replaced for Colab compatibility

from langchain.text_splitter import RecursiveCharacterTextSplitter
from helper_functions import (
                              retrieve_context_per_question,
                              replace_t_with_space,
                              show_context)
from langchain_community.embeddings import HuggingFaceEmbeddings

from evaluation.evalute_rag import evaluate_rag

from langchain.vectorstores import FAISS, Chroma

### Read Docs

In [19]:
# Download required data files
import os
os.makedirs('data', exist_ok=True)

# Download the PDF document used in this notebook
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf


--2025-11-29 22:49:55--  https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206372 (202K) [application/octet-stream]
Saving to: ‘data/Understanding_Climate_Change.pdf’

data/Understanding_ 100%[===================>] 201.54K  --.-KB/s    in 0.02s   

2025-11-29 22:49:55 (11.4 MB/s) - ‘data/Understanding_Climate_Change.pdf’ saved [206372/206372]

--2025-11-29 22:49:55--  https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185

In [20]:
path = "data/Understanding_Climate_Change.pdf"

### Encode document

In [21]:
def encode_pdf(path, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a vector store using embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A Chroma vector store containing the encoded book content.
    """

    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # Create embeddings using a local HuggingFace model
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

    # Create vector store
    vectorstore = Chroma.from_documents(cleaned_texts, embeddings)

    return vectorstore

### Other Embedding Options

Here are some other popular embedding options you can use with Langchain, along with example code snippets. Remember to install the necessary packages and set up API keys for cloud-based services.

#### 1. OpenAIEmbeddings

To use OpenAI's embedding models (e.g., `text-embedding-ada-002` or `text-embedding-3-small`), you'll need the `langchain-openai` package and your `OPENAI_API_KEY` set.

```python
# Install if not already installed
!pip install langchain-openai

from langchain_openai import OpenAIEmbeddings

# Ensure OPENAI_API_KEY is set in your environment variables
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# Or for older models: embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
```

#### 2. CohereEmbeddings

Cohere offers strong embedding models. You'll need the `langchain-cohere` package and your `COHERE_API_KEY`.

```python
# Install if not already installed
!pip install langchain-cohere

from langchain_cohere import CohereEmbeddings

# Ensure COHERE_API_KEY is set in your environment variables
embeddings = CohereEmbeddings()
```

#### 3. GoogleGenerativeAIEmbeddings (for Gemini)

If you're using Google's Gemini models, you can use `GoogleGenerativeAIEmbeddings`. You'll need `langchain-google-genai` and `GOOGLE_API_KEY`.

```python
# Install if not already installed
!pip install langchain-google-genai

from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Ensure GOOGLE_API_KEY is set in your environment variables
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
```

#### 4. BedrockEmbeddings (AWS)

For AWS users leveraging Amazon Bedrock, you can access models like `Amazon Titan Embeddings`. This requires `langchain-community` and AWS credentials configured.

```python
# Install if not already installed
!pip install langchain-community boto3

from langchain_community.embeddings import BedrockEmbeddings

# Configure AWS client (e.g., via environment variables or boto3 config)
embeddings = BedrockEmbeddings(client=boto3.client("bedrock-runtime", region_name="us-east-1"))
```

To switch to any of these, you would simply replace the `HuggingFaceEmbeddings` instantiation in your `encode_pdf` function with the chosen embedding provider's instantiation, ensuring all prerequisites (installations, API keys) are met.

In [22]:
chunks_vector_store = encode_pdf(path, chunk_size=1000, chunk_overlap=200)

/tmp/ipython-input-1140295834.py:26: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Create retriever

In [23]:
chunks_query_retriever = chunks_vector_store.as_retriever(search_kwargs={"k": 2})

### Test retriever

In [25]:
test_query = "What is the types of climate change?"
context = retrieve_context_per_question(test_query, chunks_query_retriever)
show_context(context)

Context 1:
Understanding Climate Change 
Chapter 1: Introduction to Climate Change 
Climate change refers to significant, long-term changes in the global climate. The term 
"global climate" encompasses the planet's overall weather patterns, including temperature, 
precipitation, and wind patterns, over an extended period. Over the past century, human 
activities, particularly the burning of fossil fuels and deforestation, have significantly 
contributed to climate change. 
Historical Context 
The Earth's climate has changed throughout history. Over the past 650,000 years, there have 
been seven cycles of glacial advance and retreat, with the abrupt end of the last ice age about 
11,700 years ago marking the beginning of the modern climate era and human civilization. 
Most of these climate changes are attributed to very small variations in Earth's orbit that 
change the amount of solar energy our planet receives. During the Holocene epoch, which


Context 2:
Most of these climate change

### Evaluate results

In [ ]:
#Note - this currently works with OPENAI only
evaluate_rag(chunks_query_retriever)

{'questions': ['1. **Multiple Choice: Causes of Climate Change**',
  '   - What is the primary cause of the current climate change trend?',
  '     A) Solar radiation variations',
  '     B) Natural cycles of the Earth',
  '     C) Human activities, such as burning fossil fuels',
  '     D) Volcanic eruptions',
  '',
  '2. **True or False: Climate Change Impacts**',
  '   - True or False: Climate change only affects the temperature of the planet, not weather patterns, sea levels, or ecosystems.',
  '',
  '3. **Short Answer: Mitigation Strategies**',
  '   - Describe two effective strategies that could be implemented to mitigate the effects of climate change.',
  '',
  '4. **Matching: Climate Change Terminology**',
  '   - Match the following terms with their correct definitions:',
  '     A) Greenhouse Gases',
  '     B) Carbon Footprint',
  '     C) Renewable Energy',
  '     D) Adaptation',
  '     - Definitions:',
  '       1. The total amount of greenhouse gases produced to directl